In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [2]:
bus_od = pd.read_csv(r"D:\NUS\project\LTA_data\raw_data\origin_destination_bus_202001.csv")
len(bus_od)

6835012

In [3]:
bus_stop_api = pd.read_csv(r"D:\NUS\project\LTA_data\point_data\bus_stops\busstop_from_api_202205.csv")
bus_stop_api

,Unnamed: 0,BusStopCode,RoadName,Description,Latitude,Longitude
0,0,1012,Victoria St,Hotel Grand Pacific,1.296848,103.852536
1,1,1013,Victoria St,St. Joseph's Ch,1.297710,103.853225
2,2,1019,Victoria St,Bras Basah Cplx,1.296990,103.853022
3,3,1029,Nth Bridge Rd,Opp Natl Lib,1.296673,103.854414
4,4,1039,Nth Bridge Rd,Bugis Cube,1.298208,103.855491
...,...,...,...,...,...,...
5062,5062,99139,Changi Village Rd,Blk 5,1.388195,103.987234
5063,5063,99161,Nicoll Dr,Bef Changi Beach CP 3,1.390262,103.992957
5064,5064,99171,Nicoll Dr,Changi Beach CP 2,1.391128,103.991021
5065,5065,99181,Telok Paku Rd,Bef S'pore Aviation Ac,1.387754,103.988503


In [4]:
subzone = gpd.read_file(r"D:\NUS\project\Singapore_subzone\subzone_2019\MasterPlan_SUBZONE_2019\master-plan-2019-subzone-boundary-no-sea-geojson-polygon.shp")
subzone = subzone.to_crs('epsg:4326')

In [5]:
lon = bus_stop_api.Longitude.to_list()
lat = bus_stop_api.Latitude.to_list()
pts = []
for x,y in zip(lon,lat):
    pts.append(Point(x,y))
    
bus_stops = gpd.GeoDataFrame(bus_stop_api, geometry=pts, crs="EPSG:4326")

In [6]:
subzone_bus_stops = gpd.sjoin(bus_stops, subzone, how="left", rsuffix="szon")

In [7]:
subzone_bus_stops = subzone_bus_stops[['BusStopCode','SUBZONE_N']]
subzone_bus_stops

,BusStopCode,SUBZONE_N
0,1012,VICTORIA
1,1013,VICTORIA
2,1019,BUGIS
3,1029,BUGIS
4,1039,BUGIS
...,...,...
5062,99139,CHANGI POINT
5063,99161,CHANGI AIRPORT
5064,99171,CHANGI AIRPORT
5065,99181,CHANGI POINT


In [8]:
lis = list(set(bus_od.ORIGIN_PT_CODE))
origin_code = np.unique(lis)
len(origin_code)

5008

In [9]:
commuting_volume_bus_stops = bus_od.groupby(['ORIGIN_PT_CODE','DESTINATION_PT_CODE']).sum('TOTAL_TRIPS')

In [10]:
len(commuting_volume_bus_stops)

343560

In [11]:
commuting_volume_bus_stops['od_pairs'] = commuting_volume_bus_stops.index
commuting_volume_bus_stops.index = range(len(commuting_volume_bus_stops))

In [12]:
commuting_volume_bus_stops['origin_stop'] = None
commuting_volume_bus_stops['destination_stop'] = None
for i in range(len(commuting_volume_bus_stops)):
    commuting_volume_bus_stops.loc[i,'origin_stop'] = commuting_volume_bus_stops.loc[i,'od_pairs'][0]
    commuting_volume_bus_stops.loc[i,'destination_stop'] = commuting_volume_bus_stops.loc[i,'od_pairs'][1]

In [13]:
commuting_volume_bus_stops = commuting_volume_bus_stops.drop('TIME_PER_HOUR',axis = 1)

In [152]:
commuting_volume_bus_stops.loc[commuting_volume_bus_stops['origin_stop'] == 1013].copy()

,TOTAL_TRIPS,od_pairs,origin_stop,destination_stop,Origin_Subzone,Destination_Subzone
218,393,"(1013, 1112)",1013,1112,VICTORIA,None
219,385,"(1013, 1113)",1013,1113,VICTORIA,None
220,207,"(1013, 1121)",1013,1121,VICTORIA,None
221,298,"(1013, 1211)",1013,1211,VICTORIA,None
222,428,"(1013, 1311)",1013,1311,VICTORIA,None
...,...,...,...,...,...,...
367,11,"(1013, 93011)",1013,93011,VICTORIA,None
368,4,"(1013, 93021)",1013,93021,VICTORIA,None
369,4,"(1013, 93031)",1013,93031,VICTORIA,None
370,14,"(1013, 93041)",1013,93041,VICTORIA,None


In [84]:
commuting_volume_bus_stops['Origin_Subzone'] = None
commuting_volume_bus_stops['Destination_Subzone'] = None

In [15]:
bus_stops_code = bus_stops_code.sort_values('PTcode').reset_index().drop('index',axis = 1)

In [21]:
bus_stops_code.PTcode = bus_stops_code.PTcode.astype(int)

In [40]:
bus_stops_code = bus_stops_code.fillna(0)
lis = list(set(bus_stops_code.SUBZONE_N))
subzone = np.unique(lis)

In [70]:
lis_subone = np.unique(list(subzone_bus_stops['SUBZONE_N']))
lis_bus_code = np.unique(list(subzone_bus_stops['BusStopCode']))

In [14]:
def output_and_subzone(x,df):
    bus_stops =  df.loc[df['BusStopCode'] == x].copy()
    bus_stops.index = range(len(bus_stops))
    subzone_n = bus_stops.loc[0,'SUBZONE_N']
    return subzone_n
   
def origin_bus_stops_subzone_matching(subzone_n,x):
    df = commuting_volume_bus_stops.loc[commuting_volume_bus_stops['origin_stop'] == x].copy()
    df.loc[:,'Origin_Subzone'] = subzone_n
    return df

In [179]:
def destination_bus_stops_subzone_matching(subzone_n,x):
    df = commuting_volume_bus_stops.loc[commuting_volume_bus_stops['destination_stop'] == x].copy()
    df.loc[:,'Destination_Subzone'] = subzone_n
    return df

In [186]:
del(df)
cols = commuting_volume_bus_stops.columns
df = pd.DataFrame(columns = cols)
for i in list(lis_bus_code):
    subzone_n = output_and_subzone(i,subzone_bus_stops)
    subzone_bus_stop = origin_bus_stops_subzone_matching(subzone_n,i)
    df = df.append(subzone_bus_stop)
    del(subzone_n)

C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7

C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7

C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7

C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7

C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7

...

C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7

C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7

C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7

C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7

C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3439783052.py:7

In [197]:
df2 = pd.DataFrame(columns = cols)
for x in list(lis_bus_code):
    subzone_n = output_and_subzone(x,subzone_bus_stops)
    subzone_bus_stop = destination_bus_stops_subzone_matching(subzone_n,x)
    df2 = df2.append(subzone_bus_stop)
    del(subzone_n)
df2

C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816

C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816

C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816

C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816

C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816

...

C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816

C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816

C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816

C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_29260/3836816526.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(subzone_bus_stop)


,TOTAL_TRIPS,od_pairs,origin_stop,destination_stop,Origin_Subzone,Destination_Subzone
8997,4,"(3041, 1012)",3041,1012,VICTORIA,VICTORIA
10145,1,"(3151, 1012)",3151,1012,VICTORIA,VICTORIA
10977,2,"(3223, 1012)",3223,1012,VICTORIA,VICTORIA
13581,1429,"(4142, 1012)",4142,1012,VICTORIA,VICTORIA
14521,2,"(4168, 1012)",4168,1012,VICTORIA,VICTORIA
...,...,...,...,...,...,...
340085,9,"(98011, 99189)",98011,99189,VICTORIA,CHANGI AIRPORT
340323,2,"(98021, 99189)",98021,99189,VICTORIA,CHANGI AIRPORT
341668,1,"(98109, 99189)",98109,99189,VICTORIA,CHANGI AIRPORT
342898,12,"(99041, 99189)",99041,99189,VICTORIA,CHANGI AIRPORT


In [203]:
df2['index'] = df2.index
df2 = df2.sort_values('index')
df2

,TOTAL_TRIPS,od_pairs,origin_stop,destination_stop,Origin_Subzone,Destination_Subzone,index
0,2552,"(1012, 1112)",1012,1112,VICTORIA,VICTORIA,0
1,2361,"(1012, 1113)",1012,1113,VICTORIA,BUGIS,1
2,1467,"(1012, 1121)",1012,1121,VICTORIA,ROCHOR CANAL,2
3,1694,"(1012, 1211)",1012,1211,VICTORIA,CRAWFORD,3
4,2771,"(1012, 1311)",1012,1311,VICTORIA,LAVENDER,4
...,...,...,...,...,...,...,...
343555,21,"(99189, 98239)",99189,98239,VICTORIA,LOYANG EAST,343555
343556,1,"(99189, 99019)",99189,99019,VICTORIA,CHANGI WEST,343556
343557,8,"(99189, 99029)",99189,99029,VICTORIA,CHANGI WEST,343557
343558,20,"(99189, 99039)",99189,99039,VICTORIA,CHANGI WEST,343558


In [204]:
df['Destination_Subzone'] = df2['Destination_Subzone']
df

,TOTAL_TRIPS,od_pairs,origin_stop,destination_stop,Origin_Subzone,Destination_Subzone
0,2552,"(1012, 1112)",1012,1112,VICTORIA,VICTORIA
1,2361,"(1012, 1113)",1012,1113,VICTORIA,BUGIS
2,1467,"(1012, 1121)",1012,1121,VICTORIA,ROCHOR CANAL
3,1694,"(1012, 1211)",1012,1211,VICTORIA,CRAWFORD
4,2771,"(1012, 1311)",1012,1311,VICTORIA,LAVENDER
...,...,...,...,...,...,...
343555,21,"(99189, 98239)",99189,98239,CHANGI AIRPORT,LOYANG EAST
343556,1,"(99189, 99019)",99189,99019,CHANGI AIRPORT,CHANGI WEST
343557,8,"(99189, 99029)",99189,99029,CHANGI AIRPORT,CHANGI WEST
343558,20,"(99189, 99039)",99189,99039,CHANGI AIRPORT,CHANGI WEST


In [205]:
df.to_csv(r'D:\NUS\project\LTA_data\comuting_flow\commuting_flow_202205.csv',index_label = 'ind')

In [174]:
for i in list(lis_bus_code):
    subzone_n = output_and_subzone(i,subzone_bus_stops)

In [175]:
subzone_bus_stop = origin_bus_stops_subzone_matching('CHANGI AIRPORT',i)

'CHANGI AIRPORT'

In [176]:
df = commuting_volume_bus_stops.loc[commuting_volume_bus_stops['origin_stop'] == 1013].copy()
df

,TOTAL_TRIPS,od_pairs,origin_stop,destination_stop,Origin_Subzone,Destination_Subzone
5330,343,"(2051, 1013)",2051,1013,VICTORIA,None
5857,524,"(2061, 1013)",2061,1013,VICTORIA,None
6553,59,"(2101, 1013)",2101,1013,VICTORIA,None
6804,29,"(2111, 1013)",2111,1013,VICTORIA,None
8089,498,"(3011, 1013)",3011,1013,VICTORIA,None
...,...,...,...,...,...,...
108474,13,"(28229, 1013)",28229,1013,VICTORIA,None
108752,3,"(28239, 1013)",28239,1013,VICTORIA,None
109027,82,"(28249, 1013)",28249,1013,VICTORIA,None
110235,11,"(28301, 1013)",28301,1013,VICTORIA,None


In [102]:
a.loc[0,'SUBZONE_N']

'VICTORIA'

In [118]:
a = subzone_bus_stops.loc[subzone_bus_stops['BusStopCode'] == 1012].copy()